In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "advmod"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-2-at-2023-02-23-06-36-023a1ef8.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_advmod_export_big_v3_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 18715,
 'annotations': [{'id': 2751,
   'completed_by': 1,
   'result': [{'value': {'start': 0,
      'end': 6,
      'text': 'Siiski',
      'labels': ['advmod_phrases']},
     'id': 'HZHVGbJYHa',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['free']},
     'id': 'q_1TNoqyQ4',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-02-15T13:05:05.928295Z',
   'updated_at': '2023-02-15T13:05:05.928329Z',
   'lead_time': 133.965,
   'prediction': {'id': 18715,
    'model_version': 'undefined',
    'created_ago': '2\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 0,
       'end': 6,
       'text': ['Siiski'],
       'labels': ['advmod_phrases']}}],
    'score': None,
    'cluster': None,
    'neighbor

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [10]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,18715,Train/aja_luup200106_osa_2_ud211.conllu,"Siiski jääb õhku küsimus , kas uus karistuspol...",0,6,Siiski,free,"jääb õhku küsimus , kas uus karistuspoliitika ...",86.7,93.3,93.3
1,18716,Train/aja_luup200106_osa_2_ud211.conllu,"Siiski jääb õhku küsimus , kas uus karistuspol...",27,30,kas,bound,"Siiski jääb õhku küsimus , uus karistuspoliiti...",86.7,86.7,93.3
2,18717,Train/aja_luup200106_osa_6_ud211.conllu,"Ungarlastel on keskmiselt 7 , hispaanlastel 6 ...",15,25,keskmiselt,bound,"Ungarlastel on 7 , hispaanlastel 6 partnerit .",62.5,62.5,75.0
3,18718,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",11,15,siis,free,"Aga rehmab käega : mis seal ikka , müts on ikk...",64.3,64.3,78.6
4,18719,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",28,32,seal,bound,"Aga rehmab siis käega : mis ikka , müts on ikk...",78.6,85.7,85.7
...,...,...,...,...,...,...,...,...,...,...,...
2743,21462,Train/aja_ee200110_osa_9_ud211.conllu,"Naistekas olevat selline raamat , mille kaanel...",103,105,ka,free,"Naistekas olevat selline raamat , mille kaanel...",100.0,100.0,100.0
2744,21463,Train/ilu_remsu_ud211.conllu,"Kas ristijad teadsid , et koraan keelab alkoho...",0,3,Kas,bound,"ristijad teadsid , et koraan keelab alkoholi ?",100.0,100.0,100.0
2745,21464,Train/aja_sloleht20071217_ud211.conllu,"Bell Mobility esindaja kinnitas , et firma on ...",91,95,just,free,"Bell Mobility esindaja kinnitas , et firma on ...",100.0,100.0,100.0
2746,21465,Train/aja_sloleht20071217_ud211.conllu,"Bell Mobility esindaja kinnitas , et firma on ...",96,99,nii,bound,"Bell Mobility esindaja kinnitas , et firma on ...",96.8,96.8,100.0


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,18715,Train/aja_luup200106_osa_2_ud211.conllu,"Siiski jääb õhku küsimus , kas uus karistuspol...",0,6,Siiski,free,"jääb õhku küsimus , kas uus karistuspoliitika ...",86.7,93.3,93.3
3,18718,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",11,15,siis,free,"Aga rehmab käega : mis seal ikka , müts on ikk...",64.3,64.3,78.6
6,18721,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",48,52,ikka,free,"Aga rehmab siis käega : mis seal ikka , müts o...",71.4,71.4,78.6
7,18722,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",53,58,kõige,free,"Aga rehmab siis käega : mis seal ikka , müts o...",64.3,64.3,78.6
8,18723,Train/aja_pm19980926e_ud211.conllu,Kerget sirmi on mugav kokku panna ja ümber pai...,71,76,kuigi,free,Kerget sirmi on mugav kokku panna ja ümber pai...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2739,21458,Train/aja_ee200110_osa_8_ud211.conllu,Vahel oli Soome laev ukse ees ja me saime klub...,69,74,öösel,free,Vahel oli Soome laev ukse ees ja me saime klub...,86.7,86.7,86.7
2741,21460,Train/ilu_kivirahk_ud211.conllu,Ning kopsas taas Ramile kepiga pähe .,12,16,taas,free,Ning kopsas Ramile kepiga pähe .,100.0,100.0,100.0
2742,21461,Train/aja_ee200110_osa_9_ud211.conllu,"Naistekas olevat selline raamat , mille kaanel...",94,102,võibolla,free,"Naistekas olevat selline raamat , mille kaanel...",100.0,100.0,100.0
2743,21462,Train/aja_ee200110_osa_9_ud211.conllu,"Naistekas olevat selline raamat , mille kaanel...",103,105,ka,free,"Naistekas olevat selline raamat , mille kaanel...",100.0,100.0,100.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
8,18723,Train/aja_pm19980926e_ud211.conllu,Kerget sirmi on mugav kokku panna ja ümber pai...,71,76,kuigi,free,Kerget sirmi on mugav kokku panna ja ümber pai...,100.0,100.0,100.0
9,18724,Train/aja_pm19980926e_ud211.conllu,Kerget sirmi on mugav kokku panna ja ümber pai...,71,82,kuigi hästi,free,Kerget sirmi on mugav kokku panna ja ümber pai...,100.0,100.0,100.0
14,18729,Train/aja_ml200247_osa_3_ud211.conllu,Samas on saadav tõend vaid hetkelise kehtivuse...,22,26,vaid,free,"Samas on saadav tõend hetkelise kehtivusega , ...",100.0,100.0,100.0
18,18733,Train/aja_ee199920_osa_2_ud211.conllu,Gurmaanidele tahavad lokaalid aga pakkuda mida...,30,33,aga,free,"Gurmaanidele tahavad lokaalid pakkuda midagi ,...",100.0,100.0,100.0
19,18734,Train/arborest_ud211.conllu,Arnold püsib vankumatult sadulas kõikidest pil...,13,24,vankumatult,free,Arnold püsib sadulas kõikidest pilgetest ja pü...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2734,21453,Train/aja_luup200106_osa_3_ud211.conllu,Miks mitte - eraülikool võiks teatud nishshide...,89,96,edukalt,free,Miks mitte - eraülikool võiks teatud nishshide...,100.0,100.0,100.0
2741,21460,Train/ilu_kivirahk_ud211.conllu,Ning kopsas taas Ramile kepiga pähe .,12,16,taas,free,Ning kopsas Ramile kepiga pähe .,100.0,100.0,100.0
2742,21461,Train/aja_ee200110_osa_9_ud211.conllu,"Naistekas olevat selline raamat , mille kaanel...",94,102,võibolla,free,"Naistekas olevat selline raamat , mille kaanel...",100.0,100.0,100.0
2743,21462,Train/aja_ee200110_osa_9_ud211.conllu,"Naistekas olevat selline raamat , mille kaanel...",103,105,ka,free,"Naistekas olevat selline raamat , mille kaanel...",100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [11]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,18716,Train/aja_luup200106_osa_2_ud211.conllu,"Siiski jääb õhku küsimus , kas uus karistuspol...",27,30,kas,bound,"Siiski jääb õhku küsimus , uus karistuspoliiti...",86.7,86.7,93.3
2,18717,Train/aja_luup200106_osa_6_ud211.conllu,"Ungarlastel on keskmiselt 7 , hispaanlastel 6 ...",15,25,keskmiselt,bound,"Ungarlastel on 7 , hispaanlastel 6 partnerit .",62.5,62.5,75.0
4,18719,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",28,32,seal,bound,"Aga rehmab siis käega : mis ikka , müts on ikk...",78.6,85.7,85.7
5,18720,Dev/aja_ml200247_osa6_ud211.conllu,"Aga rehmab siis käega : mis seal ikka , müts o...",33,37,ikka,bound,"Aga rehmab siis käega : mis seal , müts on ikk...",71.4,71.4,78.6
10,18725,Train/aja_pm19980926e_ud211.conllu,Kerget sirmi on mugav kokku panna ja ümber pai...,91,100,eriti kui,bound,Kerget sirmi on mugav kokku panna ja ümber pai...,87.0,87.0,91.3
...,...,...,...,...,...,...,...,...,...,...,...
2728,21447,Train/aja_pm19980926e_ud211.conllu,Kui mõlemad elektrooniliselt toimetatavad välj...,12,28,elektrooniliselt,bound,Kui mõlemad toimetatavad väljaanded on peamise...,93.3,100.0,93.3
2737,21456,Train/aja_ee200110_osa_8_ud211.conllu,Vahel oli Soome laev ukse ees ja me saime klub...,0,5,Vahel,bound,oli Soome laev ukse ees ja me saime klubi avad...,80.0,80.0,86.7
2744,21463,Train/ilu_remsu_ud211.conllu,"Kas ristijad teadsid , et koraan keelab alkoho...",0,3,Kas,bound,"ristijad teadsid , et koraan keelab alkoholi ?",100.0,100.0,100.0
2746,21465,Train/aja_sloleht20071217_ud211.conllu,"Bell Mobility esindaja kinnitas , et firma on ...",96,99,nii,bound,"Bell Mobility esindaja kinnitas , et firma on ...",96.8,96.8,100.0


In [12]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [14]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
530,19245,Train/ilu_kivirahk_ud211.conllu,Ehkki see tundus öötaeva taustal küllaltki ton...,163,168,välja,redundant comma,Ehkki see tundus öötaeva taustal küllaltki ton...,78.6,78.6,89.3
541,19256,Train/aja_horisont2000_osa_3_ud211.conllu,"Jah , tehnoloogiliselt oleme võimelised tegema...",0,3,Jah,redundant comma,", tehnoloogiliselt oleme võimelised tegema jub...",80.0,80.0,100.0
625,19340,Train/ilu_kanep_ud211.conllu,""" Ei , tead , sealt ma tulin ikka augustis ära...",2,4,Ei,redundant comma,""" , tead , sealt ma tulin ikka augustis ära , ...",93.8,93.8,100.0
792,19507,Train/aja_sloleht20071217_ud211.conllu,"Tänavu , nagu ka mullu ja tunamullu , kostitav...",0,35,"Tänavu , nagu ka mullu ja tunamullu",redundant comma,", kostitavad telekanalid vaatajaid aasta lõpus...",100.0,100.0,100.0
796,19511,Dev/aja_luup200009_ud211.conllu,"Niisiis , olgu planeedid kas või täpselt ühel ...",0,7,Niisiis,redundant comma,", olgu planeedid kas või täpselt ühel joonel ,...",70.0,76.7,83.3
990,19705,Train/aja_ee199920_osa_1_ud211.conllu,"Aga siis , kui vaene Šmigun talvel lennuki pea...",0,8,Aga siis,redundant comma,", kui vaene Šmigun talvel lennuki pealt tuli ,...",93.8,93.8,100.0
999,19714,Train/ilu_kivirahk_ud211.conllu,Imelikul kombel ei tundnud Ott mingit ärevust ...,58,69,isegi mitte,redundant comma,Imelikul kombel ei tundnud Ott mingit ärevust ...,95.0,95.0,95.0
1043,19758,Train/aja_horisont2000_osa_5_ud211.conllu,Küllap on igaüks mõistatanud koosolekute sõna-...,184,192,", et noh",redundant comma,Küllap on igaüks mõistatanud koosolekute sõna-...,80.6,80.6,100.0
1094,19809,Train/ilu_maailm_ud211.conllu,"Ei , ma ei taha süüa ega juua .",0,2,Ei,redundant comma,", ma ei taha süüa ega juua .",100.0,100.0,100.0
1170,19885,Train/aja_luup200202_osa_3_ud211.conllu,"Praegu on insaiderite kätte väga vähe jäänud ,...",28,37,väga vähe,redundant comma,"Praegu on insaiderite kätte jäänud , võibolla ...",None,None,None


In [15]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [16]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
12,18727,Train/aja_luup200106_osa_5_ud211.conllu,Seega on olukord nagu meie kahjuks .,0,5,Seega,unnatural,on olukord nagu meie kahjuks .,100.0,100.0,100.0
13,18728,Train/aja_ml200247_osa_3_ud211.conllu,Samas on saadav tõend vaid hetkelise kehtivuse...,0,5,Samas,unnatural,"on saadav tõend vaid hetkelise kehtivusega , s...",100.0,100.0,100.0
15,18730,Train/aja_horisont2000_osa_4_ud211.conllu,"Nii paneb õhuvool suulae lehvima , tegemist on...",0,3,Nii,unnatural,"paneb õhuvool suulae lehvima , tegemist ongi n...",88.9,88.9,100.0
21,18736,Train/aja_ee200110_osa_1_ud211.conllu,Samas esitasid politseinikud süüdistuse itaall...,0,5,Samas,unnatural,esitasid politseinikud süüdistuse itaallasele ...,83.3,83.3,83.3
30,18745,Train/tea_geofyysika.conllu,Kahjuks oli tähtede evolutsiooni teooria ise s...,0,7,Kahjuks,unnatural,oli tähtede evolutsiooni teooria ise sellel aj...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2714,21433,Train/aja_ml200247_osa_1_ud211.conllu,Mis teid kõige enam segab töös ?,9,14,kõige,unnatural,Mis teid enam segab töös ?,100.0,100.0,100.0
2715,21434,Train/aja_ml200247_osa_1_ud211.conllu,Mis teid kõige enam segab töös ?,9,19,kõige enam,unnatural,Mis teid segab töös ?,100.0,100.0,100.0
2732,21451,Train/aja_pm20001004_osa_2_ud211.conllu,"See tähendab , et tegelikult suhtutakse näitek...",18,28,tegelikult,unnatural,"See tähendab , et suhtutakse näiteks mustanaha...",80.0,80.0,86.7
2736,21455,Train/ilu_kivirahk_ud211.conllu,Kaugelt vaadates paistis see kõik otsekui mela...,96,99,nii,unnatural,Kaugelt vaadates paistis see kõik otsekui mela...,82.4,94.1,82.4


In [17]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [18]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1076,19791,Train/aja_epl20061216_ud211.conllu,President Toomas Hendrik Ilvese kutsel külasta...,48,52,täna,dubious,President Toomas Hendrik Ilvese kutsel külasta...,88.9,88.9,88.9
1116,19831,Train/aja_luup200106_osa_2_ud211.conllu,Näks eitab Keskvangla kinnipaneku seotust Euro...,120,124,just,dubious,Näks eitab Keskvangla kinnipaneku seotust Euro...,96.0,100.0,96.0
1296,20011,Train/tea_toohoive.conllu,Tööturg sellel perioodil pole enam mõjutatud n...,45,52,niivõrd,dubious,Tööturg sellel perioodil pole enam mõjutatud k...,85.7,85.7,92.9
1502,20218,Train/aja_pm20000218_osa_5_ud211.conllu,Veel pime asi,0,4,Veel,dubious,pime asi,100.0,100.0,100.0
1976,20693,Train/ilu_maailm_ud211.conllu,"Sugulane ei suuda siin muidugi olla , aga tema...",23,30,muidugi,dubious,"Sugulane ei suuda siin olla , aga temagi muutub .",90.0,100.0,90.0
2356,21073,Train/aja_pm19980926e_ud211.conllu,kui sealt jalutad ei tutvust teha tammega,4,9,sealt,dubious,kui jalutad ei tutvust teha tammega,100.0,100.0,100.0
2357,21074,Train/aja_pm19980926e_ud211.conllu,kui sealt jalutad ei tutvust teha tammega,18,20,ei,dubious,kui sealt jalutad tutvust teha tammega,100.0,100.0,100.0


In [19]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [20]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
342,19057,Train/ilu_remsu_ud211.conllu,Täpsemini – usuküsimus on veendumus tee õigsus...,0,9,Täpsemini,other redundant punctuation,– usuküsimus on veendumus tee õigsuses ning pr...,92.3,100.0,92.3
639,19354,Train/aja_ee200110_osa_7_ud211.conllu,"Ja nüüd : terve kogu karmitoonilist , iroonili...",0,7,Ja nüüd,other redundant punctuation,": terve kogu karmitoonilist , iroonilist , lõi...",72.2,77.8,88.9
680,19395,Train/aja_ee200110_osa_11_ud211.conllu,Nad justkui kaitseksid neid helisid välismaail...,150,163,vahest tõesti,other redundant punctuation,Nad justkui kaitseksid neid helisid välismaail...,88.0,88.0,92.0
1217,19932,Train/aja_horisont2000_osa_4_ud211.conllu,"Niisiis - norskamine on tervisele kahjulik , n...",0,7,Niisiis,other redundant punctuation,"- norskamine on tervisele kahjulik , norskamin...",100.0,100.0,100.0
1829,20545,Train/aja_ml200247_osa_5_ud211.conllu,Kolmandaks : kõiki hüvitamisele minevaid kulus...,0,10,Kolmandaks,other redundant punctuation,: kõiki hüvitamisele minevaid kulusid tuleb tõ...,90.9,90.9,100.0
2321,21038,Train/aja_luup200202_osa_1_ud211.conllu,"Esiteks - miks on neljal väikesel riigil , kel...",0,7,Esiteks,other redundant punctuation,"- miks on neljal väikesel riigil , kellest ter...",88.2,88.2,94.1


In [21]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:18:29.438041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:18:29.967396: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:18:32.149439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:18:32.149840: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data




1880it [07:09,  4.38it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")